In [1]:
import matplotlib
import numpy as np
from PIL import Image
from numpy.linalg import inv

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import ginput
from matplotlib.backend_bases import MouseButton

In [2]:
img = np.asarray(Image.open('newyork3.jpg'))

In [3]:
plt.imshow(img)

pts_src = plt.ginput(4, show_clicks=True, mouse_add=MouseButton.LEFT, mouse_pop=MouseButton.RIGHT, mouse_stop=MouseButton.MIDDLE)

pts_src1 = np.array([[1951,1800],
                    [2693,1769],
                    [2669,819],
                    [1959,1050]])

In [10]:
pts_src1 = np.round_(pts_src1)

length = max(pts_src1[:,0]) - min(pts_src1[:,0]) 
width = max(pts_src1[:,1]) - min(pts_src1[:,1]) 

pts_dst = np.array([[0,0],
                    [length,0],
                    [length,width],
                    [0,width]])



742


In [ ]:
plt.set_xlim(-5, length+5)
plt.set_ylim(-5, width+5)
plt.show()

In [ ]:
def construct_A(pts_src, pts_dst):
    num_points = pts_src.shape[0]

    A = []
    for i in range(num_points):
        rowOfPairPoints = construct_rowOfPairPoints(pts_src[i], pts_dst[i])
        np.vstack(A, rowOfPairPoints)
    return A

def construct_rowOfPairPoints(pts_src, pts_dst):
    x, y = pts_src[0], pts_src[1]
    x_t, y_t = pts_dst[0], pts_dst[1]

    A_partial = np.array([
        [-x,-y,-1,0,0,0,x_t*x,x_t*y,x_t],
        [0,0,0,-x,-y,-1,y_t*x,y_t*y,y_t]
    ])
    return A_partial